In [ ]:
import tensorflow as tf
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# train_dataset = image_dataset_from_directory(train_dir,
#                                         batch_size = batch_size,
#                                         image_size = image_size)

# validation_dataset = image_dataset_from_directory(validation_dir,                                        
#                                         batch_size = batch_size,
#                                         image_size = image_size)

# test_dataset = image_dataset_from_directory(test_dir,
#                                         batch_size = batch_size,
#                                         image_size = image_size)

In [ ]:
# EfficientNetB0/224*224/расширение данных/10 эпох

In [ ]:
conv_base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(15, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 1024)              64226304  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 15)                15375     
                                                                 
Total params: 68,291,250
Trainable params: 68,249,227
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 256

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

label_map

Found 6000 images belonging to 15 classes.
Found 1500 images belonging to 15 classes.
Found 1500 images belonging to 15 classes.


{'choco pie': 0,
 'cointreau': 1,
 'cola 0.5l': 2,
 'cola 1l': 3,
 'ermigut': 4,
 'fruto nyanya': 5,
 'granatoviy sok': 6,
 'grechka': 7,
 'hennessy': 8,
 'morozhenoe': 9,
 'pepsi 0.5l': 10,
 'pepsi 1 l': 11,
 'v baikal': 12,
 'v kalashnikov': 13,
 'v pervak': 14}

In [ ]:
train_stop = 6000//batch
validation_stop = 1500//batch
test_stop = 1500//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=validation_stop)

Epoch 1/10
23/23 [==============================] - 2748s 119s/step - loss: 1.8286 - acc: 0.5914 - val_loss: 0.3586 - val_acc: 0.8664
Epoch 2/10
23/23 [==============================] - 387s 17s/step - loss: 0.6308 - acc: 0.7819 - val_loss: 0.2785 - val_acc: 0.8969
Epoch 3/10
23/23 [==============================] - 377s 16s/step - loss: 0.4942 - acc: 0.8285 - val_loss: 0.2567 - val_acc: 0.9023
Epoch 4/10
23/23 [==============================] - 378s 17s/step - loss: 0.4716 - acc: 0.8417 - val_loss: 0.2629 - val_acc: 0.9062
Epoch 5/10
23/23 [==============================] - 375s 16s/step - loss: 0.4079 - acc: 0.8599 - val_loss: 0.2298 - val_acc: 0.9156
Epoch 6/10
23/23 [==============================] - 367s 16s/step - loss: 0.3756 - acc: 0.8644 - val_loss: 0.2409 - val_acc: 0.9086
Epoch 7/10
23/23 [==============================] - 369s 16s/step - loss: 0.3487 - acc: 0.8750 - val_loss: 0.2382 - val_acc: 0.9125
Epoch 8/10
23/23 [==============================] - 366s 16s/step - loss: 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_stop)

5/5 [==============================] - 422s 103s/step - loss: 0.2256 - acc: 0.9211


In [ ]:
from keras.models import load_model
model.save('model_1.h5')
json_string = model.to_json()

In [ ]:
# Дообучение

In [ ]:
name = 'top_conv'

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == name:
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 1024)              64226304  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 15)                15375     
                                                                 
Total params: 68,291,250
Trainable params: 64,653,839
Non-trainable params: 3,637,411
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=validation_stop)

Epoch 1/20
23/23 [==============================] - 400s 17s/step - loss: 0.3236 - acc: 0.9009 - val_loss: 0.2405 - val_acc: 0.9133
Epoch 2/20
23/23 [==============================] - 382s 17s/step - loss: 0.2787 - acc: 0.9088 - val_loss: 0.2384 - val_acc: 0.9219
Epoch 3/20
23/23 [==============================] - 370s 16s/step - loss: 0.2569 - acc: 0.9131 - val_loss: 0.2098 - val_acc: 0.9258
Epoch 4/20
23/23 [==============================] - 383s 17s/step - loss: 0.2377 - acc: 0.9142 - val_loss: 0.2095 - val_acc: 0.9289
Epoch 5/20
23/23 [==============================] - 376s 16s/step - loss: 0.2201 - acc: 0.9222 - val_loss: 0.2248 - val_acc: 0.9250
Epoch 6/20
23/23 [==============================] - 369s 16s/step - loss: 0.2301 - acc: 0.9231 - val_loss: 0.2120 - val_acc: 0.9242
Epoch 7/20
23/23 [==============================] - 376s 16s/step - loss: 0.2347 - acc: 0.9189 - val_loss: 0.2357 - val_acc: 0.9250
Epoch 8/20
23/23 [==============================] - 383s 17s/step - loss: 0.

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_stop)

5/5 [==============================] - 57s 11s/step - loss: 0.2070 - acc: 0.9328


In [ ]:
model.save('molel_2.h5')

In [ ]:
# EfficientNetV2L

In [ ]:
conv_base = tf.keras.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

473186304/473176280 [==============================] - 11s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetv2-l"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                   

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(15, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              64226304  
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 15)                15375     
                                                                 
Total params: 181,988,527
Trainable params: 181,475,951
Non-trainable params: 512,576
__________________________________

In [ ]:
conv_base.trainable = False

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              64226304  
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 15)                15375     
                                                                 
Total params: 181,988,527
Trainable params: 64,241,679
Non-trainable params: 117,746,848
_______________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=validation_stop)

Epoch 1/20
23/23 [==============================] - 500s 21s/step - loss: 2.3385 - acc: 0.5172 - val_loss: 0.7351 - val_acc: 0.7453
Epoch 2/20
18/23 [======================>.......] - ETA: 1:17 - loss: 0.8631 - acc: 0.7110